In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

In [2]:
from disease_symptoms import dataset
data = dataset

rash joint pain muscle pain headache
chikungunya


In [3]:
diseases_list = []
symptoms_list = []
for disease, symptoms in data.items():
    diseases_list.append(disease)
    for sym in symptoms:
        symptoms_list.append(sym)

In [4]:
data = {'text' : diseases_list + symptoms_list, 'label' : ['disease'] * len(diseases_list) + ['symptoms']*len(symptoms_list)}

In [5]:
X_train = data['text']
Y_train = data['label']

In [6]:
label_to_id = {'disease' : 0, 'symptoms' : 1}
Y_train = [label_to_id[label] for label in Y_train]

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train)

In [8]:
import joblib
joblib.dump(tokenizer,'entity_identification_tokenizer.joblib')

['entity_identification_tokenizer.joblib']

In [9]:
num_classes = len(set(Y_train))
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=2)

In [10]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X_train.shape[1]),
    LSTM(100),
    Dense(2, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(np.array(X_train), np.array(Y_train_one_hot), epochs=100)



Epoch 1/100
8/8 [==============================] - 2s 6ms/step - loss: 0.6826 - accuracy: 0.7336
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6423 - accuracy: 0.8035
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5731 - accuracy: 0.8035
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4841 - accuracy: 0.8035
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4771 - accuracy: 0.8035
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.4473 - accuracy: 0.8035
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4314 - accuracy: 0.8035
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: 0.4025 - accuracy: 0.8035
Epoch 9/100
8/8 [==============================] - 0s 5ms/step - loss: 0.3722 - accuracy: 0.8035
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.3357 - accuracy: 0.8079
Epoch 11/100
8/8 [===========

In [13]:
model.save('entity_detection_model')

INFO:tensorflow:Assets written to: entity_detection_model\assets


INFO:tensorflow:Assets written to: entity_detection_model\assets


### Test Model

In [14]:
import joblib
loaded_model= tf.keras.models.load_model('entity_detection_model')
tokenizer = joblib.load('entity_identification_tokenizer.joblib')
user_input = "Tell me about dengue"

user_input_sequence = tokenizer.texts_to_sequences([user_input])
user_input_padded = tf.keras.preprocessing.sequence.pad_sequences(user_input_sequence, maxlen=4)

In [15]:
prediction = loaded_model(user_input_padded)

In [16]:
if prediction[0][0] > prediction[0][1]:
    print("The input is related to a disease.")
else:
    print("The input is related to symptoms.")

The input is related to a disease.


In [17]:
user_input = "I am feeling dizziness and vomiting"
user_input_sequence = tokenizer.texts_to_sequences([user_input])
user_input_padded = tf.keras.preprocessing.sequence.pad_sequences(user_input_sequence, maxlen=4)
prediction = loaded_model(user_input_padded)
if prediction[0][0] > prediction[0][1]:
    print("The input is related to a disease.")
else:
    print("The input is related to symptoms.")

The input is related to symptoms.
